In [37]:
import os
import time
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

In [2]:
path = 'data'
df = pd.read_csv(os.path.join(path, 'target.csv'))
print(f"Row : {df.shape[0]}, \t Columns : {df.shape[1]}")
df.head()

Row : 13522, 	 Columns : 3


,isbn13,title,description
0,9788982412615,IBM 통합(Intergrated) 시스템,현장 실무자를 위한 IBM 서버 안내서. IBM AS/400D을 기능에 초점을 맞추...
1,9788988557211,(단계별 실습에 의한)델파이 프로그래밍=Delphi programming,"비주얼형 객체지향 프로그래밍 언어로서 비중러 베이직, 비주얼 C++, 자바 등과 함..."
2,9788982412707,컴퓨터 활용 실습,컴퓨터를 잘 활용하기 위하여 반드시 알아 두어야 할 기초적인 내용으로 컴퓨터를 전공...
3,9788982412578,(아주 특별한)비주얼베이직6 게임과 프로젝트,중급자를 위한 비주얼 베이직6 교재이다. 비주얼 베이직 6를 활용한 게임 프로그래밍...
4,9788982412691,컴퓨터활용의 첫걸음:컴퓨터의 기초 윈도우즈와 인터넷 한글,컴퓨터를 잘 활용하기 위하여 반드시 알아야 할 기초적인 내용으로 컴퓨터를 전공하지 ...


In [33]:
df['text'] = df['title'] + ' '+ df['description']


In [16]:
def user_tokenizer(text):
    okt = Okt()
    return okt.nouns(text)

In [34]:
stopwords = ['등','때','때문','또한','로','로서','를','먼저','및','별', '별로','뿐','의', '이','판','편집', '평가','후''개정',
             '개정판','거나','것', '게','결','곧바로', '곳', '곳곳', '공','군더더기','그', '그간', '그것', '그대로', '그동안',
             '꼭','년','낼','내','나','늘','단','달', '달리','더', '더욱', '데','도','또', '뜻','란','론', '률', '리','링',
             '마치','몇','몸', '못','문',]

In [35]:
vectorizer = TfidfVectorizer(tokenizer=user_tokenizer, min_df=0.001, stop_words=stopwords)
start = time.time()
feature_vect = vectorizer.fit_transform(df['description']+df['title'])
print(f"Process End in {time.time()-start}")
print(feature_vect.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['개정', '후'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Process End in 164.36038732528687
(13522, 1937)


In [76]:
cosine_matrix = cosine_similarity(feature_vect)


In [101]:
def recommender(df, cosine_matrix, isbn, rank=10, show_detail=False):
    target_idx = df[df['isbn13'] == isbn].index[0]
    sorted_idx = cosine_matrix[target_idx].argsort()[::-1][1:rank+1]
    reconmmends = df[df.index.isin(sorted_idx[:rank])]
    print('Target Data')
    print(f"Title : {df[df.index == target_idx]['title'].values[0]}")
    print('--'*50)
    print()
    for i, idx in enumerate(sorted_idx):
        print(f"Rank : {i+1}")
        print(f"ISBN : {df[df.index == idx]['isbn13'].values[0]}")
        print(f"Title : {df[df.index == idx]['title'].values[0]}")
        if show_detail == True :
            print(f"Desc : {df[df.index == idx]['description'].values[0]}")
            print(f"Cosine Smilarity : {cosine_matrix[target_idx][idx]}")
            
        print("=="*50)


In [102]:
df['isbn13'].tolist()

['9788982412615',
 '9788988557211',
 '9788982412707',
 '9788982412578',
 '9788982412691',
 '9788931418385',
 '9788931418613',
 '9788984620773',
 '9788984620346',
 '9788973542949',
 '9788986525915',
 '9788973542659',
 '9788973542697',
 '9788973542901',
 '9788973542895',
 '9788973542932',
 '9788973542918',
 '9788973542963',
 '9788973542789',
 '9788973542666',
 '9788986525946',
 '9788986525939',
 '9788955021196',
 '9788973542772',
 '9788973542734',
 '9788986525922',
 '9788989109297',
 '9788973542758',
 '9788973542840',
 '9788973542925',
 '9788973542727',
 '9788981611286',
 '9788973542673',
 '9788989109280',
 '9788989366089',
 '9788973542826',
 '9788955021219',
 '9788955021202',
 '9788973542796',
 '9788955021295',
 '9788983970923',
 '9788931418163',
 '9788931546996',
 '9788988886946',
 '9788940690352',
 '9788988886915',
 '9788931418286',
 '9788931547078',
 '9788985966542',
 '9788931418217',
 '9788988886953',
 '9788931547054',
 '9788931418408',
 '9788985966559',
 '9788931417951',
 '97889314

In [106]:
recommender(df = df, cosine_matrix = cosine_matrix, isbn='9788931547078', rank=10, show_detail=True)

Target Data
Title : (for webmaster)홈페이지 제작 실전 테크닉
----------------------------------------------------------------------------------------------------

Rank : 1
ISBN : 9788933604670
Title : (21세기)홈페이지 만들기
Desc : 인터넷이란 무엇인지 그 기능과 이해를 시작으로 쉽고 빠르게 인터넷 홈페이지를 만드는 방법을 설명한 지침서. 홈페이지 기획에서부터 나만의 홈페이지 만들기,멀티미디어로 홈페이지 멋내기,동영상 넣기,홈페이지에 자바 스크립트 응용하기까지를 소개했다.
Cosine Smilarity : 0.6661546707538674
Rank : 2
ISBN : 9788933212011
Title : 홈페이지 만들기:playing book
Desc : 이 책은 초보자를 위한 홈페이지 교재이다. 나모 웹에디터 4의 설치에서부터 홈페이지 제작과 등록, 그리고 관리에 이르기까지 홈페이지를 만들기 위해 필요한 것들을 다루었다.
Cosine Smilarity : 0.6045900062788975
Rank : 3
ISBN : 9788981610425
Title : (내 마음대로 요리하는)홈페이지 300
Desc : 홈페이지 제작 지침서. 홈페이지에 대한 기초지식부터 홈페이지 제작에 필요한 프로그램, 홈페이지 제작의 실제까지를 상세하게 설명했다. 부록 CD-ROM 포함.
Cosine Smilarity : 0.5720709608221949
Rank : 4
ISBN : 9788940678169
Title : (멀티미디어)홈페이지 만들기:최대의 활용서
Desc : 홈페이지의 입문에서 중급까지 알기 쉽게 설명한 홈페이지 제작 해설서. 홈페이지의 감을 잡자를 소개한 1장을 시작으로 나모 웹에디터를 이용한 홈페이지 만들기, 포토샵과 페인트샵 프로 해설,웹 프로그래밍과 멀티미디어 홈페이지 만들기 등을 설명했다.
Cosine Smilari